# Exercise: Simplex pivots in the short tableau
<font color='blue'><b>Goal:</b></font>
Writing a function to perform one pivot step in the short tableau.

<font color='blue'><b>Required packages:</b></font>`numpy`

Suppose you are given the following LP from the lecture notes:

$$\begin{equation}\label{eq:sampleTermLp}
\begin{array}{rrcrcll}
\max z = &400 x_1 &+ &900 x_2 &       &    & \\
         &x_1     &+ &4 x_2   & \leq  &40  & \text{(constraint 1)} \\
         &2x_1    &+ & x_2    & \leq  &42  & \text{(constraint 2)}\\
         &1.5x_1  &+ &3x_2    & \leq  &36  & \text{(constraint 3)}\\
         &x_1     &  &        & \geq  &0   & \text{(non-negativity for $x_1$)}\\
         &        &  &x_2     & \geq  &0   & \text{(non-negativity for $x_2$)}\\
\end{array}
\end{equation}$$
This problem's graphical presentation looks as follows:

![](pivoting_fig1.png)

After adding slack variables, you obtain the short tableau
$$\begin{array}{c|rr|r}
&x_1&x_2&1\\
\hline
z&-400&-900&0\\
\hline
y_1&1&4&40\\
y_2&2&1&42\\
y_3&1.5&3&36
\end{array}$$

We will implement a function to perform pivoting steps on short tableaus as for example the one above.

## Additional `numpy` functionality
For your task, you will need to perform row operations. When working with matrices in `numpy`, this can be done as follows. Let us first create a matrix $A$:

In [71]:
import numpy as np
A = np.array(
[
    [1,2],
    [3,4],
    [5,6]
])
A

array([[1, 2],
       [3, 4],
       [5, 6]])

To obtain the first row, we type

In [72]:
A[0,:]

array([1, 2])

Subtracting the first row from the second, for example, can be done with the code

In [73]:
A[1,:] = A[1,:] - A[0,:]
A

array([[1, 2],
       [2, 2],
       [5, 6]])

To get the number of rows of $A$, type

In [74]:
A.shape[0]

3

Analogously, we can select columns as follows:

In [75]:
A[:,0]

array([1, 2, 5])

To get the number of columns of $A$, type

In [76]:
A.shape[1]

2

Finally, to obtain an entry $A_{i,j}$, type (exemplified for $i=0$, $j=1$):

In [77]:
a_01 = A[0,1]
a_01

2

Furthermore, you may need to append a standard unit vector $v$ to a matrix $A$. This can be done as follows, where we exemplify the procedure with the first standard unit vector $e_1$:

In [79]:
v = np.array(np.zeros((A.shape[0],1)))
v[0,0] = 1 # first standard unit vector
A = np.append(A, v, axis=1)
A

array([[1., 2., 1., 1.],
       [2., 2., 0., 0.],
       [5., 6., 0., 0.]])

Finally, to delete a column $i$, let's say for $i=0$ (the first column), type:

In [83]:
A = np.delete(A,[0],1)
A

array([], shape=(3, 0), dtype=float64)

## A pivoting function
We now want to implement a function `pivot(T, i, j)` which takes in a short tableau $T$, and replaces the basis variable corresponding to row $i$ of the tableau by the variable corresponding to column $j$ of the tableau. By convention, the tableau $T$ should include the objective function row as the first row (the row with index $0$), and the right hand sides as the last column.

To avoid accidental selection of a pivot from the objective function row or the column of right-hand sides, your function should check that the input indices $i$ and $j$ are indeed pointing at a potential pivot. If not, the function should print a message saying so, and return the input tableau. Similarly, if the indicated pivot element is zero (in which case the pivoting step cannot be performed), the function should print a message saying so, and return the input tableau.
If a non-zero pivot is given, the pivoting step should be carried out, and the new tableau should be returned. Additionally, the function should print a message indicating whether or not the pivot was legal for phase II of the simplex method.

Complete the given framework below to obtain the pivoting function with the desired functionality.

In [99]:
def pivot(T, i, j):
	rhsIndex = T.shape[1] - 1

	if (0 == i  # objective function row
			or j == rhsIndex  # rhs
	):
		print("[Error] Given indices are not pointing at a potential pivot.")
		return T

	if (0 == T[i, j]):  # Check if the indicated pivot is zero
		print("[Error] Given pivot is zero, step cannot be performed.")
		return T

	def quotient(row):
		return T[row, rhsIndex] / T[row, j]

	# check quotient rule
	isMinQuotient = True
	chosenQutient = quotient(i)
	for row in range(T.shape[0]):
		if (T[row, j] <= 0):
			continue
		if (quotient(row) < chosenQutient):
			isMinQuotient = False
			break

	if (T[0, j] < 0 and isMinQuotient):  # Check if indicated pivot is legal for phase II of Simplex Method
		print("[Info] Given pivot is legal.")
	else:
		print("[Warning] Given pivot is illegal.")

	# Implement pivoting step here
	T_pivoted = np.copy(T)

	for row in range(T_pivoted.shape[0]):
		for col in range(T_pivoted.shape[1]):
			# pivot element
			if (i == row and j == col):
				T_pivoted[row, col] = 1 / T[i, j]
			# pivot col
			elif (j == col):
				T_pivoted[row, col] = -T[row, col] / T[i, j]
			# pivot row
			elif (i == row):
				T_pivoted[row, col] = T[row, col] / T[i, j]
			# other elements
			else:
				T_pivoted[row, col] = T[row, col] - (T[row, j] * T[i, col]) / T[i, j]

	return T_pivoted

## Testing your implementation
To check whether your implementation works, use your function to reproduce the following pivot steps from the lecture notes:

![](pivoting_fig2.png)

This can be done, for example, by running the code below:

In [102]:
import numpy as np
T = np.array([
    [-400,-900,0],
    [1,4,40],
    [2,1,42],
    [1.5,3,36]
])

T = pivot(T,2,0)
print("Vertex b")
print(T)

T = pivot(T,3,1)
print("Vertex c")
print(T)

T = pivot(T,1,0)
print("Vertex d")
print(T)

[Info] Given pivot is legal.
Vertex b
[[ 2.00e+02 -7.00e+02  8.40e+03]
 [-5.00e-01  3.50e+00  1.90e+01]
 [ 5.00e-01  5.00e-01  2.10e+01]
 [-7.50e-01  2.25e+00  4.50e+00]]
[Info] Given pivot is legal.
Vertex c
[[-3.33333333e+01  3.11111111e+02  9.80000000e+03]
 [ 6.66666667e-01 -1.55555556e+00  1.20000000e+01]
 [ 6.66666667e-01 -2.22222222e-01  2.00000000e+01]
 [-3.33333333e-01  4.44444444e-01  2.00000000e+00]]
[Info] Given pivot is legal.
Vertex d
[[ 5.00000000e+01  2.33333333e+02  1.04000000e+04]
 [ 1.50000000e+00 -2.33333333e+00  1.80000000e+01]
 [-1.00000000e+00  1.33333333e+00  8.00000000e+00]
 [ 5.00000000e-01 -3.33333333e-01  8.00000000e+00]]


Next, let us check whether your function gives a warning if the indicated pivot is not legal. To this end let us take another look at the graphical represenation of the original LP:

![](pivoting_fig1.png)

We see that the first and the second constraint are never both tight for any point in the polytope. In other words, there is no solution with $y_1=y_2=0$.

Let us check whether your function detects that the pivoting step taking us to the solution with $y_1=y_2=0$ is illegal! To this end, note that we get to this solution from the tableau for "verted $d$" above if we pivot such that $y_2$ exits the basis and $y_3$ enters the basis. The following call executes precisely the pivoting operation corresponding to this basis change (assuming that you didn't change $T$ after the above pivoting steps).

In [98]:
T = pivot(T,1,1)
print(T)

[Warning] Given pivot is illegal.
[[ 2.00000000e+02  1.00000000e+02  1.22000000e+04]
 [-6.42857143e-01 -4.28571429e-01 -7.71428571e+00]
 [-1.42857143e-01  5.71428571e-01  1.82857143e+01]
 [ 2.85714286e-01 -1.42857143e-01  5.42857143e+00]]


Now, let us see whether your function correctly gives an error if a pivot operation cannot be performed. To this end, consider the following short tableau (which is unrelated to the LP above) taken from the script, and try to replace $y_1$ by $y_2$.

$$\begin{array}{c|rr|c}
 &y_3&y_2&1\\
\hline
z&1&1&3\\
\hline
y_1&1&0&3\\
x_2&2&-1&3\\
x_1&1&-1&1
\end{array}$$

In [103]:
# Apply your function to the tableau above.
T = np.array([
    [1,1,3],
    [1,0,3],
    [2,-1,3],
    [1,-1,1]
])
T = pivot(T,1,1)

[Error] Given pivot is zero, step cannot be performed.


Finally, you can apply your pivoting functions to other examples from class. By doing tests as the above ones, try to see whether the different cases are recognised correctly by your function!